# Introduction 
L'objectif du notebook, comparé à la semaine dernière est de rajouter des indicateurs pertinents en modifiant la liste précédemment établie :  

- **marketCap** - Capitalisation boursière
- **marginProfit** - Marge nette
- **roe** - Rentabilité des fonds propres (%)
- **roic** - Rentabilité des capitaux investis (%)
- **revenuePerShare** - Chiffre d'affaires par action
- **netIncomePerShare** - Bénéfice net par action
- **debtToEquity** - D/E Ratio
- **peRatio** - P/E Ratio
- **shareValue** - Valeur de l'action  

Et d'ajouter des indicateurs de croissance annuelle dans les cas où ils sont pertients :   
- **marginProfitYoYGrowth** - Croissance annuelle de la marge nette (%)
- **roeYoYGrowth** - Croissance annuelle du ROE (%)
- **roicYoYGrowth** - Croissance annuelle du ROIC (%)
- **revenuePerShareYoYGrowth** - Croissance annuelle du chiffre d'affaires par action (%)
- **netIncomePerShareYoYGrowth** - Croissance annuelle des bénéfices nets (%)
- **debtToEquityYoYGrowth** - Croissance annuelle du D/E Ratio (%)

Ces indicateurs seront évalués sous la forme de time series sur 5 ans de 2020 à 2024.

In [1]:
# Import des librairies et données
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")

price = pd.read_csv('../csv/historical_ohlc_nasdaq100.csv')
fundamental = pd.read_csv('../csv/nasdaq100_fundamentals_fpm.csv')

display(price.head())
display(fundamental.head())

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,symbol
0,2020-01-02,330.00,334.48,329.170,334.43,334.429993,1990496,1990100,4.43,1.34000,332.94,"January 02, 20",0.013400,ADBE
1,2020-01-03,329.17,332.98,328.690,331.81,331.809998,1579371,1577600,2.64,0.80202,331.84,"January 03, 20",0.008020,ADBE
2,2020-01-06,328.29,333.91,328.200,333.71,333.709991,1875122,1874700,5.42,1.65000,332.42,"January 06, 20",0.016500,ADBE
3,2020-01-07,334.15,334.79,332.305,333.39,333.390000,2507261,2500800,-0.76,-0.22744,333.80,"January 07, 20",-0.002274,ADBE
4,2020-01-08,333.81,339.23,333.410,337.87,337.869995,2248531,2248500,4.06,1.22000,337.20,"January 08, 20",0.012200,ADBE


,symbol,date,calendarYear,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,...,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare
0,ADBE,2024-11-29,2024,FY,48.109620,12.438479,18.022371,17.503356,17.642058,31.554810,...,337500000.0,0.0,35.167635,55.879983,0.0,10.378861,6.531856,0.0,0.394186,0.519016
1,ADBE,2023-12-01,2023,FY,42.470460,11.877462,15.978118,15.190372,17.159737,36.144420,...,346500000.0,0.0,41.823896,48.687341,0.0,8.727068,7.496815,0.0,0.328611,0.787746
2,ADBE,2022-12-02,2022,FY,37.459574,10.119149,16.676596,15.736170,12.970213,29.895745,...,345500000.0,0.0,42.810690,63.896074,0.0,8.525908,5.712401,0.0,0.338481,0.940426
3,ADBE,2021-12-03,2021,FY,33.092243,10.109015,15.142558,14.450734,12.155136,31.020964,...,309000000.0,0.0,43.425404,61.061662,0.0,8.405218,5.977564,0.0,0.325877,0.691824
4,ADBE,2020-11-27,2020,FY,26.752599,10.935551,11.906445,11.035343,12.457380,27.575884,...,257500000.0,0.0,39.654181,64.860627,0.0,9.204578,5.627451,0.0,0.396562,0.871102


In [2]:
fundamental

,symbol,date,calendarYear,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,...,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare
0,ADBE,2024-11-29,2024,FY,48.109620,12.438479,18.022371,17.503356,17.642058,31.554810,...,337500000.0,0.0,35.167635,55.879983,0.000000,10.378861,6.531856,0.000000,0.394186,0.519016
1,ADBE,2023-12-01,2023,FY,42.470460,11.877462,15.978118,15.190372,17.159737,36.144420,...,346500000.0,0.0,41.823896,48.687341,0.000000,8.727068,7.496815,0.000000,0.328611,0.787746
2,ADBE,2022-12-02,2022,FY,37.459574,10.119149,16.676596,15.736170,12.970213,29.895745,...,345500000.0,0.0,42.810690,63.896074,0.000000,8.525908,5.712401,0.000000,0.338481,0.940426
3,ADBE,2021-12-03,2021,FY,33.092243,10.109015,15.142558,14.450734,12.155136,31.020964,...,309000000.0,0.0,43.425404,61.061662,0.000000,8.405218,5.977564,0.000000,0.325877,0.691824
4,ADBE,2020-11-27,2020,FY,26.752599,10.935551,11.906445,11.035343,12.457380,27.575884,...,257500000.0,0.0,39.654181,64.860627,0.000000,9.204578,5.627451,0.000000,0.396562,0.871102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,ZS,2024-07-31,2024,FY,14.491804,-0.385771,5.213362,3.910460,16.108820,8.517522,...,20895000.0,57913500.0,124.013600,17.831205,0.000000,2.943226,20.469733,0.000000,-0.045292,1.302903
500,ZS,2023-07-31,2023,FY,11.155855,-1.395972,3.189848,2.301741,14.490155,5.002774,...,22317500.0,101018500.0,131.520379,18.591428,116.519091,2.775235,19.632704,3.132534,-0.279040,0.888107
501,ZS,2022-07-31,2022,FY,7.742972,-2.769992,2.284765,1.641875,12.288151,4.068988,...,19350500.0,71791500.0,133.743490,39.401235,129.876136,2.729105,9.263669,2.810370,-0.680757,0.642890
502,ZS,2021-07-31,2021,FY,4.961888,-1.931598,1.489377,1.059630,11.076356,3.898853,...,8890000.0,44806500.0,139.421758,30.466647,139.313218,2.617956,11.980314,2.619995,-0.495427,0.429748


# Merging

In [3]:
# Conversion des colonnes date en type datetime
price.date = pd.to_datetime(price.date)
fundamental.date = pd.to_datetime(fundamental.date)

# Ajout des valeurs 'close' (valeur à la cloture) à fundamental
df = fundamental.merge(price[['symbol', 'date', 'close']], on = ['symbol', 'date'], how='left')
df.close.isna().sum()

# On voit que trop de valeurs (la moitié) sont manquantes

254

In [4]:
# Trie les DF pour merge_asof
fundamental = fundamental.sort_values(by=['date', 'symbol'])
price = price.sort_values(by=['date', 'symbol'])

# Merging à la date la plus proche pour éviter les valeurs manquantes
df = pd.merge_asof(fundamental, 
                   price[['symbol', 'date', 'close']], # On garde uniquement les colonnes qui nous intéressent
                   on='date',
                   by='symbol',
                   direction='backward')

# Cleaning

In [5]:
# Gestion des cas particuliers de valeurs manquantes restantes
# price[price.symbol =='CCEP'] - date la plus proche (3 jours de décalage)
df.loc[0, 'close'] = 50.93

# price[price.symbol =='PDD'] -  date la plus proche (3 jours de décalage)
df.loc[1, 'close'] = 41.23

# price[price.symbol =='APP'] - Pas de valeur proche - entrée en bourse plus tard
df.loc[37, 'close'] = 65.20   # On ajoute, dans ce cas, tout de même la valeur de l'entrée en bourse 4 mois plus tard

# price[price.symbol =='CEG'] -  Pas de valeur proche pour l'année 2020, on ajoute à 2021
df.loc[147, 'close'] = 42.00 # 19 jours d'écart

# price[price.symbol =='GEHC'] # Pas de valeur proche
# price[price.symbol =='ARM']  # Pas de valeur proche
# Pour le reste, on supprimmera les NaNs

# On supprimme Alphabet catégorie C
df = df[df.symbol != 'GOOG']

# On crée marginProfit
df['marginProfit'] = df.netIncomePerShare / df.revenuePerShare

# On renomme close pour refléter la variable dans le contexte du df fundamental
df = df.rename(columns={'close':'shareValue'})
columns_of_interest = ['symbol',
                       'calendarYear', 
                       'marketCap', 
                       'marginProfit', 
                       'roe',
                       'roic',
                       'revenuePerShare',
                       'netIncomePerShare',
                       'debtToEquity',
                       'peRatio',
                       'shareValue']

clean_df = df[columns_of_interest]

# On supprimme les valeurs NaNs (Entreprises entrées en bourses plus tard)
clean_df = clean_df.dropna()

In [6]:
# On remarque des outliers au niveau de min / max par rapport au Q1 et Q3 (IQR)
clean_df.describe()

,calendarYear,marketCap,marginProfit,roe,roic,revenuePerShare,netIncomePerShare,debtToEquity,peRatio,shareValue
count,492.000000,4.920000e+02,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000
mean,2022.048780,2.239000e+11,0.115875,0.264621,0.122093,44.796351,5.916388,0.535395,-167.145055,229.830935
std,1.444724,6.000375e+11,0.275173,2.929921,0.172418,80.382119,12.748441,10.433174,5460.294237,375.383423
min,2019.000000,1.603054e+09,-2.943927,-32.589389,-1.309052,0.609703,-29.574113,-194.549374,-117858.733542,6.420000
25%,2021.000000,3.645430e+10,0.070630,0.057754,0.043943,12.846238,1.121912,0.182910,16.718417,66.227500
50%,2022.000000,5.496283e+10,0.148548,0.187318,0.109118,23.786916,3.534687,0.494927,27.749101,135.755000
75%,2023.000000,1.388596e+11,0.231964,0.332212,0.197180,43.013659,7.384398,1.216585,46.505323,253.210000
max,2025.000000,6.031718e+12,0.864708,53.038744,1.969265,706.055559,174.944977,43.952473,26277.621897,4968.420000


In [7]:
# On affiche la distribution des variables
indicators = ['marketCap', 
              'marginProfit', 
              'roe',
              'roic',
              'revenuePerShare',
              'netIncomePerShare',
              'debtToEquity',
              'peRatio',
              'shareValue']

fig = make_subplots(rows = 3, cols = 3)
for i, indicator in enumerate(indicators):
    box = px.strip(clean_df[indicator], 
                   hover_name=clean_df['symbol']) 
    
    # Division integer et modulo pour indiquer les rows/cols dans la boucle     
    fig.add_trace(box.data[0], row=i//3+1, col=(i%3)+1) 

fig.update_layout(title='Distribution des variables', 
                  width=1300, 
                  height=900)\
                  .show(renderer='iframe') # à changer en .show('notebook') ou .show() si non fonctionnel

In [8]:
# On retire les outliers car sûrement dûs à des erreurs dans la donnée
symbols_to_filter = ['PDD', 'MELI', 'VRSK', 'MDB', 'ORLY', 'MSTR']
filter_series = ~clean_df['symbol'].isin(symbols_to_filter)
clean_df = clean_df[filter_series]

In [9]:
# On crée 5 lignes, une par année pour l'indice NASDAQ 
# et comparer des entreprises individuelles à la moyenne pondérée de l'indice 

years = [2020, 2021, 2022, 2023, 2024] 

# Fonction de calcul de la moyenne pondérée
def mean_nasdaq(metric, df):
    return (df[metric] * df.marketCap).sum() / df.marketCap.sum()

for year in years:
    nsdq = clean_df[clean_df.calendarYear == year]
    
    n_data = {
    'symbol'            : ['NASDAQ'],
    'calendarYear'      : year, 
    'marketCap'         : [mean_nasdaq('marketCap', nsdq)], 
    'marginProfit'      : [mean_nasdaq('marginProfit', nsdq)], 
    'roe'               : [mean_nasdaq('roe', nsdq)],
    'roic'              : [mean_nasdaq('roic', nsdq)],
    'revenuePerShare'   : [mean_nasdaq('revenuePerShare', nsdq)],
    'netIncomePerShare' : [mean_nasdaq('netIncomePerShare', nsdq)],
    'debtToEquity'      : [mean_nasdaq('debtToEquity', nsdq)],
    'peRatio'           : [mean_nasdaq('peRatio', nsdq)],
    'shareValue'        : [mean_nasdaq('shareValue', nsdq)]}
    
    nasdaq_df = pd.DataFrame(n_data, columns=columns_of_interest)
    clean_df = pd.concat([clean_df, nasdaq_df], axis=0)

In [10]:
# On vérifie le bon ajout des valeurs
display(clean_df.isna().sum())
clean_df

symbol               0
calendarYear         0
marketCap            0
marginProfit         0
roe                  0
roic                 0
revenuePerShare      0
netIncomePerShare    0
debtToEquity         0
peRatio              0
shareValue           0
dtype: int64

,symbol,calendarYear,marketCap,marginProfit,roe,roic,revenuePerShare,netIncomePerShare,debtToEquity,peRatio,shareValue
0,CCEP,2019,2.114228e+10,0.090705,0.176550,0.080541,25.712871,2.332282,1.043047,19.452923,50.930000
2,LULU,2019,3.212492e+10,0.162239,0.330699,0.229307,30.517712,4.951155,0.379036,49.760103,239.390000
3,EA,2020,2.934981e+10,0.548853,0.407318,0.319443,18.897611,10.372014,0.133494,9.657720,100.170000
4,MCHP,2020,1.619742e+10,0.108187,0.102157,0.149314,11.038510,1.194224,1.697646,28.386646,33.900000
5,TTWO,2020,1.341432e+10,0.130937,0.159283,0.126726,27.312814,3.576245,0.069803,33.166073,118.610000
...,...,...,...,...,...,...,...,...,...,...,...
0,NASDAQ,2020,8.965772e+11,0.172628,0.321242,0.148748,30.984720,4.223135,0.975355,94.333044,190.790349
0,NASDAQ,2021,1.124789e+12,0.213482,0.448293,0.197215,35.520955,6.178423,0.868810,46.720408,232.759278
0,NASDAQ,2022,9.487101e+11,0.208751,0.590143,0.215430,41.614554,6.360706,1.246786,14.854333,175.594318
0,NASDAQ,2023,1.213803e+12,0.210058,0.422199,0.203884,45.389697,7.761861,0.671955,39.816742,249.077927


# Ajout des indicateurs de croissance

In [11]:
# Tri du df
df_sorted = clean_df.sort_values(by=['symbol', 'calendarYear'])

# Chaque indicateur est comparé à sa version année n+1 et un ratio est fait pour en retirer la croissance annuelle
for indicator in indicators: # indicators définit au moment de l'affichage de la distribution
    yoy_growth_col_name = f'{indicator}YoYGrowth'
    df_sorted[yoy_growth_col_name] = df_sorted.groupby('symbol')[indicator].pct_change() * 100

In [12]:
df_sorted = df_sorted.reset_index().drop('index', axis=1)

In [13]:
df_sorted

,symbol,calendarYear,marketCap,marginProfit,roe,roic,revenuePerShare,netIncomePerShare,debtToEquity,peRatio,shareValue,marketCapYoYGrowth,marginProfitYoYGrowth,roeYoYGrowth,roicYoYGrowth,revenuePerShareYoYGrowth,netIncomePerShareYoYGrowth,debtToEquityYoYGrowth,peRatioYoYGrowth,shareValueYoYGrowth
0,AAPL,2020,1.948296e+12,0.209136,0.878664,0.242695,15.820258,3.308587,1.871440,33.935934,112.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAPL,2021,2.453751e+12,0.258818,1.500713,0.389251,21.903541,5.669029,2.163925,25.916254,146.92,25.943439,23.755735,70.794977,60.386812,38.452490,71.342897,15.628867,-23.631825,30.851443
2,AAPL,2022,2.439367e+12,0.253096,1.969589,0.451748,24.317273,6.154614,2.614462,24.441824,150.43,-0.586187,-2.210638,31.243507,16.055739,11.019825,8.565579,20.820369,-5.689210,2.389055
3,AAPL,2023,2.695570e+12,0.253062,1.560760,0.433892,24.344473,6.160669,1.787533,27.790812,171.21,10.502825,-0.013459,-20.757053,-3.952602,0.111853,0.098379,-31.629037,13.701876,13.813734
4,AAPL,2024,3.495160e+12,0.239713,1.645935,0.443071,25.484915,6.109054,1.872327,37.287278,227.79,29.663136,-5.275295,5.457269,2.115500,4.684604,-0.837818,4.743635,34.171246,33.047135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,ZS,2020,1.679259e+10,-0.266924,-0.237436,-0.080770,3.334821,-0.890143,1.867128,-145.875391,129.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
463,ZS,2021,3.200214e+10,-0.389287,-0.495427,-0.134253,4.961888,-1.931598,1.823812,-122.132036,235.91,90.572938,45.841902,108.656922,66.216386,48.790264,116.998552,-2.319953,-16.276464,81.678860
464,ZS,2022,2.162738e+10,-0.357743,-0.680757,-0.190299,7.742972,-2.769992,1.824040,-55.415326,155.06,-32.418939,-8.103061,37.408067,41.746616,56.048891,43.404155,0.012494,-54.626707,-34.271544
465,ZS,2023,2.324580e+10,-0.125134,-0.279040,-0.121431,11.155855,-1.395972,1.669459,-114.887676,160.38,7.483178,-65.021348,-59.010389,-36.189167,44.077182,-49.603744,-8.474620,107.321124,3.430930


In [14]:
# On supprimme ceux qui ne sont pas pertinents sous la forme croissance annuelle
df_sorted = df_sorted.drop(['marketCapYoYGrowth', 'peRatioYoYGrowth', 'shareValueYoYGrowth'], axis=1)

In [15]:
# Enregistrement du csv
df_sorted.to_csv('/kaggle/working/clean_nasdaq_data_opa.csv', index=False)

# Comparaison du P/E Ratio et de la croissance annuelle des bénéfices 
Une croissance plus élevée qu'un P/E ratio peut indiquer une opportunité boursière.  
On remarque une certaine linéarité dans les graphs

In [16]:
# Ticker à analyser - remplacer pour choisir l'entreprise
company_symbol = 'ZS'
df_company = df_sorted[df_sorted['symbol'] == company_symbol].copy()

# Création d'une figure avec un second axe y 
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Ajout d'un graph pour le P/E Ratio
fig.add_trace(
    go.Scatter(x=df_company['calendarYear'], y=df_company['peRatio'], name="P/E Ratio", mode='lines+markers'),
    secondary_y=False,
)

# Ajout d'un graph pour le EPS Growth
fig.add_trace(
    go.Scatter(x=df_company['calendarYear'], y=df_company['netIncomePerShareYoYGrowth'], name="Croissance annuelle EPS (%)", mode='lines+markers'),
    secondary_y=True,
)

# Mise en page
fig.update_layout(
    title_text=f"{company_symbol} - Tendance du P/E Ratio vs. Croissance de l'EPS"
)
fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="<b>P/E Ratio</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Croissance annuelle EPS (%)</b>", secondary_y=True)
fig.update_xaxes(dtick=1)

fig.show(renderer='iframe') # à changer en .show('notebook') ou .show() si non fonctionnel

**Graphique "Croissance des Bénéfices (EPS) vs. Ratio P/E"**


**Le marché suit-il les profits ?**  
Normalement, si les profits (EPS) augmentent, le P/E devrait aussi monter (le marché valorise mieux l'action).  

**L'action est-elle trop ou pas assez chère ?**  
Profits qui montent + P/E bas : L'action est peut-être sous-évaluée.  

Profits qui montent peu (ou baissent) + P/E élevé : L'action est peut-être surévaluée.  

**L'optimisme du marché change-t-il ?**  
Si le P/E monte plus vite que les profits, le marché devient plus optimiste (il paie plus cher pour chaque euro de bénéfice).

Si le P/E baisse alors que les profits montent (ou baissent moins vite), le marché devient plus pessimiste.

**Le marché a-t-il confiance ?**  
Si la croissance des profits et le P/E sont tous les deux élevés et stables, cela suggère que le marché a confiance dans l'avenir de l'entreprise.

En Bref : Ce graphique aide à voir si le prix de l'action (via le P/E) est justifié par la croissance réelle des profits (EPS), et comment le sentiment du marché évolue par rapport aux résultats de l'entreprise.

Attention : Si une entreprise ne fait pas de profit (EPS négatif), le P/E devient négatif ou non calculable, et le graphique est plus difficile à lire à ce moment-là.

# Visualisation de l'ensemble des timeseries du dataset
Visualisation interactive

In [17]:
# Définition des métriques pour l'affichage interactif
metrics_to_choose = ['marketCap', 
                     'marginProfit', 
                     'peRatio', 
                     'roe', 
                     'roic',
                     'debtToEquity',
                     'shareValue',
                     'marginProfitYoYGrowth',
                     'roeYoYGrowth',
                     'roicYoYGrowth',
                     'revenuePerShareYoYGrowth',
                     'netIncomePerShareYoYGrowth',
                     'debtToEquityYoYGrowth']							


# Dictionnaire pour les labels et titres associés
metric_labels = {
    'marketCap'                  : 'Capitalisation boursière',
    'marginProfit'               : 'Marge nette',
    'peRatio'                    : 'P/E Ratio',
    'roe'                        : 'Rentabilité des fonds propres (%)',
    'roic'                       : 'Rentabilité des capitaux investis (%)',
    'debtToEquity'               : 'D/E Ratio',
    'shareValue'                 : "Valeur de l'action",
    'marginProfitYoYGrowth'      : 'Croissance annuelle de la marge nette (%)',
    'roeYoYGrowth'               : 'Croissance annuelle du ROE (%)',
    'roicYoYGrowth'              : 'Croissance annuelle du ROIC (%)',
    'revenuePerShareYoYGrowth'   : "Croissance annuelle du chiffre d'affaires par action (%)",
    'netIncomePerShareYoYGrowth' : 'Croissance annuelle des bénéfices nets (%)',
    'debtToEquityYoYGrowth'      : 'Croissance annuelle du D/E Ratio (%)',
}

In [18]:
worse = clean_df.sort_values(by='peRatio', ascending=True)['symbol'].unique()[:5].tolist()
best = clean_df.sort_values(by='peRatio', ascending=False)['symbol'].unique()[:5].tolist() 
tocompare = worse + best + ['NASDAQ', 'META', 'AAPL']

In [19]:
# --- Préparation des données ---
companies = tocompare # Ajouter tous les tickers désirés
filtering = df_sorted['symbol'].isin(companies)
df_to_plot = df_sorted[filtering].copy()

# NVDA est décalé d'un an (bilans en janvier)
condition = df_to_plot['symbol'] == 'NVDA'
df_to_plot.loc[condition, 'calendarYear'] -= 1 # Réaligne Nvidia

# --- Création du Graphique Initial ---
initial_metric = metrics_to_choose[0]
initial_label = metric_labels[initial_metric]

fig = px.line(
    df_to_plot, 
    x='calendarYear',
    y=initial_metric,
    color='symbol', 
    title=f"Comparaison: {initial_label} de 2020 à 2024",
    labels={initial_metric: initial_label},
    markers=True,
    height=600
)

# --- Création des Boutons pour le Menu de Métriques ---
buttons_metric = []
for metric in metrics_to_choose:
    label = metric_labels[metric]
    y_data_update = [df_to_plot[df_to_plot['symbol'] == company][metric].tolist() for company in companies]
    hover_template = f"<b>{label}</b>: %{{y}}<br>Year: %{{x}}<br>Symbol: %{{fullData.name}}<extra></extra>"
    buttons_metric.append(
        dict(
            method='update',
            label=label,
            args=[
                {'y': y_data_update, 
                 'hovertemplate': [hover_template] * len(companies)},
                {'title.text': f"Comparaison: {label} de 2020 à 2024",
                 'yaxis.title.text': label}
            ]
        )
    )

# --- Ajout du Menu et Mise en Page ---
fig.update_layout(
    updatemenus=[
        dict(
            # Menu pour les métriques
            buttons=buttons_metric,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.8, xanchor="left",
            y=1.15, yanchor="top"
        )
    ],
    xaxis_title="Year",
    yaxis_title=initial_label,
    legend_title_text='Tickers (Cliquer pour afficher/masquer)',
    showlegend=True
)

fig.update_xaxes(dtick=1)
fig.show(renderer='iframe')

# Suite du projet 
Il conviendra ensuite de se pencher sur une analyse métier des graphs ci dessus ainsi qu'un comparatif de corrélation des time series

In [78]:
url = 'https://en.wikipedia.org/wiki/Nasdaq-100'
table = pd.read_html(url)
ndx_returns = table[3] 
ndx_returns = ndx_returns.drop(['Closing level', 'Change in Index in Points'], axis =1)
ndx_returns = ndx_returns[ndx_returns.Year > 2018]
ndx_returns.columns = ['Year', 'Annual_Returns']
ndx_returns['Annual_Returns'] = ndx_returns['Annual_Returns'].astype(str).str.replace('−', '-', regex=False)
ndx_returns.Annual_Returns = pd.to_numeric(ndx_returns.Annual_Returns)

In [86]:
df_sorted['shareValue_NY'] = df_sorted.groupby('symbol')['shareValue'].shift(-1)
df_sorted['return%'] = ((df_sorted['shareValue_NY'] / df_sorted['shareValue'])-1) * 100

benchmark = ndx_returns.set_index('Year')['Annual_Returns']
df_sorted['benchmark'] = df_sorted['calendarYear'].map(benchmark)
df_sorted['target'] = np.where((df_sorted['return%'] > df_sorted['benchmark']), 1, 0)
df_final = df_sorted.drop(['return%', 'shareValue_NY', 'benchmark'], axis=1)


In [98]:
df_final['index'] = df_final.symbol + '_' + df_final.calendarYear.astype('string')
df_final['index']

df_modeling = df_final.drop(['symbol', 'calendarYear'], axis=1).set_index('index')
df_modeling

,marketCap,marginProfit,roe,roic,revenuePerShare,netIncomePerShare,debtToEquity,peRatio,shareValue,marginProfitYoYGrowth,roeYoYGrowth,roicYoYGrowth,revenuePerShareYoYGrowth,netIncomePerShareYoYGrowth,debtToEquityYoYGrowth,target
index,,,,,,,,,,,,,,,,
AAPL_2020,1948295921320.00,0.21,0.88,0.24,15.82,3.31,1.87,33.94,112.28,NaN,NaN,NaN,NaN,NaN,NaN,0
AAPL_2021,2453750882240.00,0.26,1.50,0.39,21.90,5.67,2.16,25.92,146.92,23.76,70.79,60.39,38.45,71.34,15.63,0
AAPL_2022,2439367314090.00,0.25,1.97,0.45,24.32,6.15,2.61,24.44,150.43,-2.21,31.24,16.06,11.02,8.57,20.82,1
AAPL_2023,2695569789510.00,0.25,1.56,0.43,24.34,6.16,1.79,27.79,171.21,-0.01,-20.76,-3.95,0.11,0.10,-31.63,0
AAPL_2024,3495160329570.00,0.24,1.65,0.44,25.48,6.11,1.87,37.29,227.79,-5.28,5.46,2.12,4.68,-0.84,4.74,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZS_2020,16792591550.00,-0.27,-0.24,-0.08,3.33,-0.89,1.87,-145.88,129.85,NaN,NaN,NaN,NaN,NaN,NaN,1
ZS_2021,32002135140.00,-0.39,-0.50,-0.13,4.96,-1.93,1.82,-122.13,235.91,45.84,108.66,66.22,48.79,117.00,-2.32,0
ZS_2022,21627382500.00,-0.36,-0.68,-0.19,7.74,-2.77,1.82,-55.42,155.06,-8.10,37.41,41.75,56.05,43.40,0.01,1


In [99]:
px.histogram(df_modeling, x='target', color='target').update_xaxes(dtick=1).show()

In [100]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 

X = df_modeling.drop('target', axis=1)
y= df_modeling.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
# Créer et configurer le classificateur TPOT
tpot = TPOTClassifier(
    generations=3,            # Nombre de générations pour la recherche 
    population_size=20,       # Taille de la population (nombre de modèles par génération)
    cv=5,                     # Validation croisée à 5 plis
    random_state=42,          # Assure la reproductibilité
    verbosity=2               # Niveau de détails affichés durant l'exécution
)

tpot.fit(X_train, y_train)
y_pred = tpot.predict(X_test)
accuracy_score(y_test, y_pred)


Version 0.12.1 of tpot is outdated. Version 1.0.0 was released Wednesday February 26, 2025.


Imputing missing values in feature set


Optimization Progress:   0%|          | 0/80 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7319279279279279

Generation 2 - Current best internal CV score: 0.7319279279279279

Generation 3 - Current best internal CV score: 0.7319279279279279

Best pipeline: GradientBoostingClassifier(input_matrix, learning_rate=0.01, max_depth=6, max_features=0.5, min_samples_leaf=5, min_samples_split=16, n_estimators=100, subsample=1.0)
Imputing missing values in feature set


0.6702127659574468

In [107]:
best_pipeline = tpot.fitted_pipeline_
final_estimator = best_pipeline.steps[-1][1]
importances = final_estimator.feature_importances_
feature_names = X.columns
pd.DataFrame({'Features':feature_names, 'Importances':importances}).sort_values(by='Importances', ascending=False)

,Features,Importances
0,marketCap,0.15
4,revenuePerShare,0.10
14,debtToEquityYoYGrowth,0.10
12,revenuePerShareYoYGrowth,0.09
9,marginProfitYoYGrowth,0.08
3,roic,0.07
7,peRatio,0.06
8,shareValue,0.06
2,roe,0.06
1,marginProfit,0.05


In [108]:
from sklearn.metrics import classification_report
print(classification_report(y_true = y_test, y_pred = y_pred))

              precision    recall  f1-score   support

           0       0.67      0.98      0.79        61
           1       0.75      0.09      0.16        33

    accuracy                           0.67        94
   macro avg       0.71      0.54      0.48        94
weighted avg       0.70      0.67      0.57        94



In [110]:
methods = ['pearson', 
           'spearman']

titles = ['Linéaire', 
          'Monotonique']
cols_to_plot = ['marketCap', 
                'marginProfit', 
                'roe', 
                'debtToEquity', 
                'peRatio', 
                'pbRatio', 
                'dividendYield', 
                'shareValue', 
                'test_indicator', 
                'shareValue_NY', 
                '%Return_NY']

fig = make_subplots(rows = 1, cols = 2)
for i, (method, title) in enumerate(zip(methods, titles)):
    heatmap = px.imshow(df_modeling.corr(method=method)\
                                .round(2), 
                                text_auto=True,
                                color_continuous_scale='Plasma',
                                title=title)
    
    fig.add_trace(heatmap.data[0], row=1, col=i+1)
    
fig.update_layout(title='Matrices de corrélation features au global / rendement en %', 
                  coloraxis=dict(colorscale='Plasma'), 
                  width= 1300, 
                  annotations=[dict(text=titles[i],
                               x=0.18 + 0.65*i,
                               y=1.1,
                               xref="paper", 
                               yref="paper",
                               showarrow=False) for i in range(2)]).show(renderer='iframe')

In [23]:
pd.options.display.float_format = '{:.2f}'.format

In [24]:
cols_to_check = ['peRatio', 'roic', 'debtToEquity', 'roe', 'marginProfit']
q1 = df_sorted[cols_to_check].quantile(0.25)
q3 = df_sorted[cols_to_check].quantile(0.75)

IQR = q3 - q1 
threshold_pos = q3 + (10 * IQR)
threshold_neg = q1 - (10 * IQR)
threshold_pos

peRatio        341.88
roic             1.73
debtToEquity    11.14
roe              3.07
marginProfit     1.77
dtype: float64

In [25]:
df_filtered = df_sorted[((df_sorted[cols_to_check] <= threshold_pos).all(axis=1)) & ((df_sorted[cols_to_check] >= threshold_neg).all(axis=1))]
df_filtered

,symbol,calendarYear,marketCap,marginProfit,roe,roic,revenuePerShare,netIncomePerShare,debtToEquity,peRatio,shareValue,marginProfitYoYGrowth,roeYoYGrowth,roicYoYGrowth,revenuePerShareYoYGrowth,netIncomePerShareYoYGrowth,debtToEquityYoYGrowth,shareValue_NY,return%
0,AAPL,2020,1948295921320.00,0.21,0.88,0.24,15.82,3.31,1.87,33.94,112.28,NaN,NaN,NaN,NaN,NaN,NaN,146.92,-23.58
1,AAPL,2021,2453750882240.00,0.26,1.50,0.39,21.90,5.67,2.16,25.92,146.92,23.76,70.79,60.39,38.45,71.34,15.63,150.43,-2.33
2,AAPL,2022,2439367314090.00,0.25,1.97,0.45,24.32,6.15,2.61,24.44,150.43,-2.21,31.24,16.06,11.02,8.57,20.82,171.21,-12.14
3,AAPL,2023,2695569789510.00,0.25,1.56,0.43,24.34,6.16,1.79,27.79,171.21,-0.01,-20.76,-3.95,0.11,0.10,-31.63,227.79,-24.84
4,AAPL,2024,3495160329570.00,0.24,1.65,0.44,25.48,6.11,1.87,37.29,227.79,-5.28,5.46,2.12,4.68,-0.84,4.74,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,XEL,2024,38013760000.00,0.14,0.10,0.05,23.87,3.44,1.51,19.64,67.52,15.54,-1.36,2.93,-7.23,7.18,-3.21,NaN,NaN
462,ZS,2020,16792591550.00,-0.27,-0.24,-0.08,3.33,-0.89,1.87,-145.88,129.85,NaN,NaN,NaN,NaN,NaN,NaN,235.91,-44.96
463,ZS,2021,32002135140.00,-0.39,-0.50,-0.13,4.96,-1.93,1.82,-122.13,235.91,45.84,108.66,66.22,48.79,117.00,-2.32,155.06,52.14
464,ZS,2022,21627382500.00,-0.36,-0.68,-0.19,7.74,-2.77,1.82,-55.42,155.06,-8.10,37.41,41.75,56.05,43.40,0.01,160.38,-3.32


In [26]:
merged_df_index = df_sorted.merge(
    df_filtered,
    how='left',        # Keep all rows from df_large
    left_index=True,
    right_index=True,
    indicator=True,    # Add the '_merge' column
    suffixes=('', '_small'))


df_diff_merge = merged_df_index[merged_df_index['_merge'] == 'left_only']

# Drop the extra columns added by the merge ('_merge' and any suffixed cols)
# Identify columns original to df_large
original_cols = df_sorted.columns
df_diff_merge = df_diff_merge[original_cols] # Keep only original columns

df_diff_merge

,symbol,calendarYear,marketCap,marginProfit,roe,roic,revenuePerShare,netIncomePerShare,debtToEquity,peRatio,shareValue,marginProfitYoYGrowth,roeYoYGrowth,roicYoYGrowth,revenuePerShareYoYGrowth,netIncomePerShareYoYGrowth,debtToEquityYoYGrowth,shareValue_NY,return%
6,ABNB,2021,102557840000.00,-0.06,-0.07,0.07,9.73,-0.57,0.51,-291.36,166.49,-95.67,-95.33,-110.50,-18.12,-96.46,-36.93,85.50,94.73
34,AEP,2024,48890447138.56,0.15,0.76,0.04,37.20,5.60,11.72,16.48,92.23,29.34,768.84,15.42,1.70,31.54,578.53,NaN,NaN
52,AMZN,2022,855876000000.00,-0.01,-0.02,0.02,50.44,-0.27,0.96,-314.43,84.00,-107.46,-107.72,-76.65,8.66,-108.10,13.95,151.94,-44.72
60,APP,2020,23338686146.80,-0.09,0.79,0.01,4.05,-0.35,-10.68,-185.32,65.20,NaN,NaN,NaN,NaN,NaN,NaN,94.26,-30.83
61,APP,2021,35355923073.60,0.01,0.02,0.02,7.45,0.09,1.55,997.46,94.26,-114.62,-97.91,41.87,83.69,-126.86,-114.49,10.53,795.16
65,ARM,2024,130392806486.11,0.09,0.06,0.02,3.15,0.30,0.04,426.12,124.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,AXON,2020,7570148460.00,-0.00,-0.00,-0.00,11.02,-0.03,0.02,-4391.04,122.53,NaN,NaN,NaN,NaN,NaN,NaN,157.00,-21.96
82,AZN,2021,166520418549.20,0.00,0.00,-0.01,26.39,0.08,0.78,1486.79,58.25,-97.51,-98.61,-106.30,30.07,-96.76,-39.91,67.80,-14.09
91,BKNG,2020,91260160980.00,0.01,0.01,0.01,165.86,1.44,2.53,1546.78,2227.27,NaN,NaN,NaN,NaN,NaN,NaN,2399.23,-7.17
140,CRWD,2021,46819717560.00,-0.11,-0.11,-0.05,4.02,-0.43,0.89,-505.45,215.80,NaN,NaN,NaN,NaN,NaN,NaN,180.64,19.46


In [27]:
df_sorted.loc['NASDAQ']

KeyError: 'NASDAQ'